In [4]:
import os
os.chdir('/...')  
print(os.getcwd())

/...

In [5]:
from src.models.nn.moe.resnet_block_moe import ResNetBlockMoE
from src.models.nn.moe.routing import GlobalAvgLinearRoutingNetwork
from src.models.nn.moe.resnet_conv_moe import ResNetConvMoE

MOE_TYPE = ResNetBlockMoE

In [7]:
import torchinfo
from torchinfo import summary


def get_models(k=1):
    models = [
        MOE_TYPE(
            routing_layer_type=GlobalAvgLinearRoutingNetwork,
            layers=18,
            num_experts=ne,
            k=k if k >= 1 else max(round(ne * k), 1),
        )
        for ne in (1, 2, 4, 8, 16, 32)
    ]
    return models


params = [summary(model).total_params for model in get_models(k=1)]


def multi_adds(k=1):
    models = get_models(k)
    return [summary(model, input_size=(1, 1, 28, 28)).total_mult_adds for model in models]

ModuleNotFoundError: No module named 'torchinfo'

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(params, multi_adds(k=0.99), marker="o")
plt.plot(params, multi_adds(k=0.5), marker="o")
plt.plot(params, multi_adds(k=2), marker="o")
plt.plot(params, multi_adds(k=1), marker="o")
plt.legend(["k = #Experts", "k = 0.5 #Experts", "k = 2", "k = 1"])
# plt.xlim(0)
# plt.ylim(0)
plt.ylabel("FLOPs")
plt.xlabel("Parameters")
plt.xscale("log")
plt.yscale("log")
plt.savefig("resnet_block_moe_flops_params.png")